#### hypotheses

#### If add maybe work well
- add distance between airports
- add meteo
- add holidays for **from** ans **to**
- mean ticket prices 24h before, avrage before

##### Data reading

In [1]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn import set_config
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('./data/flights_train.csv.bz2')
df_sub = pd.read_csv('./data/flights_Xtest.csv.bz2')

In [3]:
df['flight_date'] = pd.to_datetime(df['flight_date'])
df_sub['flight_date'] = pd.to_datetime(df_sub['flight_date'])

In [4]:
df_sub.head()

,flight_date,from,to,avg_weeks,std_weeks
0,2013-01-22,ATL,MCO,10.363636,8.232025
1,2013-02-22,ATL,DEN,8.294118,5.542616
2,2013-01-31,ORD,EWR,8.933333,9.407191
3,2012-11-15,DFW,ORD,9.900000,6.608607
4,2013-02-13,SEA,SFO,7.000000,4.472136


In [5]:
df.head()

,flight_date,from,to,avg_weeks,target,std_weeks
0,2012-06-19,ORD,DFW,12.875000,12.331296,9.812647
1,2012-09-10,LAS,DEN,14.285714,10.775182,9.466734
2,2012-10-05,DEN,LAX,10.863636,11.083177,9.035883
3,2011-10-09,ATL,ORD,11.480000,11.169268,7.990202
4,2012-02-21,DEN,SFO,11.450000,11.269364,9.517159


In [6]:
df.shape

(8896, 6)

In [7]:
df.describe()

,avg_weeks,target,std_weeks
count,8896.000000,8896.000000,8896.000000
mean,11.580564,11.079129,8.648275
std,2.726443,0.949174,2.045742
min,2.625000,5.586968,2.160247
25%,9.722222,10.528622,7.216347
50%,11.447222,11.063493,8.597628
75%,13.320000,11.630678,10.079555
max,22.372093,14.224277,14.958061


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8896 entries, 0 to 8895
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   flight_date  8896 non-null   datetime64[ns]
 1   from         8896 non-null   object        
 2   to           8896 non-null   object        
 3   avg_weeks    8896 non-null   float64       
 4   target       8896 non-null   float64       
 5   std_weeks    8896 non-null   float64       
dtypes: datetime64[ns](1), float64(3), object(2)
memory usage: 417.1+ KB


In [9]:
# df = df.set_index('flight_date')

#### Pipeline

In [10]:
numeric_preprocessor = Pipeline(steps=[("scaler", StandardScaler())])

In [11]:
categorical_preprocessor = Pipeline(steps=[("onehot", OneHotEncoder(handle_unknown="ignore"))])

In [12]:
preprocessor = ColumnTransformer(
    [
        ("categorical", categorical_preprocessor, ["from", "to"]),
        ("numerical", numeric_preprocessor, ["avg_weeks", "std_weeks"]),
    ]
)

In [13]:
pipe = make_pipeline(preprocessor, RandomForestRegressor(n_estimators=100))

In [14]:
y = df.target
X = df.drop(['target', 'flight_date'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=42)

In [15]:
# pipe.fit(X, y)
pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['from', 'to']),
                                                 ('numerical',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['avg_weeks',
                                                   'std_weeks'])])),
                ('randomforestregressor', RandomForestRegressor())])

In [16]:
y_pred = pipe.predict(X_test)

In [17]:
mse(y_test, y_pred)**0.5

#####  TO csv

In [18]:
# df_sub = df_sub.drop(['flight_date'], axis=1)
# submition = pd.DataFrame(pipe.predict(df_sub))

In [19]:
# from datetime import datetime
# path = 'submition_' + datetime.now().strftime('%Y-%m-%d_%H-%M-%S') + '.csv'
# submition.to_csv(path, index=False, header=False)